# Random Walk Sampling

In this notebook, we will compute the Random Walk from a set of seeds using cuGraph.  


| Author Credit |    Date    |  Update        | cuGraph Version |  Test Hardware |
| --------------|------------|----------------|-----------------|----------------|
| Brad Rees     | 04/20/2021 | created        | 0.19            | GV100, CUDA 11.0
| Ralph Liu     | 06/22/2022 | updated/tested | 22.08           | TV100, CUDA 11.5
| Don Acosta    | 08/28/2022 | updated/tested | 22.10           | TV100, CUDA 11.5

Currently NetworkX does not have a random walk function.  There is code on StackOverflow that generates a random walk by getting a vertex and then randomly selecting a neighbor and then repeating the process.  

### Test Data
We will be using the Zachary Karate club dataset 
*W. W. Zachary, An information flow model for conflict and fission in small groups, Journal of
Anthropological Research 33, 452-473 (1977).*


<img src="../../img/zachary_black_lines.png" width="35%"/>


Because the test data has vertex IDs starting at 1, the auto-renumber feature of cuGraph (mentioned above) will be used so the starting vertex ID is zero for maximum efficiency. The resulting data will then be auto-unrenumbered, making the entire renumbering process transparent to users.

In [ ]:
#  Import the modules
import cugraph
import cudf

# Import a built-in dataset
from cugraph.experimental.datasets import karate

In [ ]:
gdf = karate.get_edgelist(fetch=True)

In [ ]:
gdf['weight'] = 1.0

In [ ]:
# Create a Graph - using the source (src) and destination (dst) vertex pairs from the Dataframe 
G = cugraph.Graph()
G.from_cudf_edgelist(gdf, source='src', destination='dst', edge_attr='weight')

In [ ]:
# some stats on the graph
(G.number_of_nodes(), G.number_of_edges() )

In [ ]:
# create a list with the seeds
seeds = [17,19]

In [ ]:
# random walk path length
path_length = 4

In [ ]:
rw, so, sz = cugraph.random_walks(G, "uniform", seeds, path_length, use_padding=True)

A random walk generates a path from a seed vertex. At each step on the random walk (starting from the seed), the random walker picks a random departing edge to traverse. The random walk will terminate in two situations, when the maximum path length is reached, or when the current vertex on the path has no departing edges to traverse. The result of a single random walk will be a path of some length less than or equal to the maximum path length.

cugraph.random_walks performs a random walk from each of the specified seeds. The output will be a path for each of the seeds. Because the path lengths might be variable length, the return value consists of a pair of outputs.

The first output provides the edges used on the paths.

The second output represents the seed offset, which is a cuDF Series. The seed offset identifies the offset of the first entry in the first output for a particular seed.

In [ ]:
rw.head(10)

In [ ]:
idx = 0
for i in range(len(seeds)):
    for j in range(path_length):
        print(f"{rw[idx]}", end=" ")
        idx += 1
    print(" ")

-----
Copyright (c) 2022, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
